<a href="https://colab.research.google.com/github/yuxiao-ashes339/My-Wedding-Optimizer/blob/main/WeddingOptimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Wedding Cost Optimizer
# Initial data from https://www.theknot.com/content/wedding-budget-ways-to-save-money
